#Simple Model Training with Darts Library

In [2]:
#work with darts library to create a time series forecast
import darts
import pandas as pd



from darts.models.forecasting.rnn_model import RNNModel


#types = ["LSTM", "GRU", "RNN"] available models
choosen_model = "LSTM"
chunk_length = 10
output_chunk_length = 1
n_epochs = 100

#rnn model 
model = RNNModel(model=choosen_model, input_chunk_length=chunk_length, output_chunk_length=output_chunk_length, n_epochs=n_epochs, random_state=0)  

